In [2]:
import sys
sys.path.append('../..')

from pymongo import ASCENDING as A
from tqdm import tqdm

from config import client
import sql_parser

In [5]:
KEYS = {
    'osu_beatmap_difficulty': [("beatmap_id", A), ("mods", A)],
    'osu_beatmap_difficulty_attribs': [("beatmap_id", A), ("mods", A), ("attrib_id", A)],
    'osu_beatmap_failtimes': [("beatmap_id", A), ("type", A)],
    'osu_user_beatmap_playcount': [("user_id", A), ("beatmap_id", A)],
}

In [ ]:
sql_parser.parse_dump(
    path = '../../../data/2021_04_03_performance_mlpp_osu_random/',
    db = client['osu_mlpp_db1'],
    keys=KEYS
)

In [ ]:
sql_parser.parse_dump(
    path = '../../../data/2021_04_01_performance_osu_top/',
    db = client['osu_top_db'],
    toMigrate=[
        'osu_user_stats',
        'osu_scores_high',
        'osu_user_beatmap_playcount',
        'sample_users'
    ],
    keys=KEYS
)

In [ ]:
from pymongo import UpdateOne
import numpy as np

In [6]:
osu_db = client['osu_mlpp_db1']

In [7]:
osu_db.osu_scores_high.create_index('user_id')
osu_db.osu_scores_high.create_index('beatmap_id')
osu_db.osu_scores_high.create_index('date')
osu_db.osu_scores_high.create_index([('user_id', A), ('date', A)])

'user_id_1_date_1'

In [8]:
def bonus_table():
    cnt_bonus = []
    curr = 1
    for i in range(100000):
        curr *= .9994
        cnt_bonus.append(416.6667 * (1 - curr))
    return np.array(cnt_bonus)

In [9]:
bonus = bonus_table()
user_ids = osu_db.osu_user_stats.find().distinct('_id')

In [ ]:
for user_id in tqdm(user_ids):
    scores = list(osu_db.osu_scores_high.find(
        {'user_id': user_id}, {'pp': 1, 'date': 1}).sort('date'))
    score_hist = list(map(lambda x: x['pp'] if x['pp'] else 0, scores))
    pp_hist = np.array(fast_pp_hist(score_hist))
    bonus_pp_hist = pp_hist + bonus[:len(pp_hist)]

    score_updates = []
    for i in range(len(scores)):
        query = {'_id': scores[i]['_id']}
        update = {
            '$set': {
                'mlpp.est_user_pp_raw': pp_hist[i],
                'mlpp.est_user_pp': bonus_pp_hist[i]
            }
        }
        score_updates.append(UpdateOne(query, update))

    osu_db.osu_scores_high.bulk_write(score_updates)

    osu_db.osu_user_stats.update_one({'_id': user_id}, {
        '$set': {
            'mlpp.est_current_pp_raw': pp_hist[-1],
            'mlpp.est_current_pp': bonus_pp_hist[-1]
        }
    })

In [ ]:
import os

from config import client
from beatmap_parser.beatmap_parser.beatmap import parse_hit_objects
from beatmap_parser.beatmap_parser.errors import FileFormatError

In [ ]:
osu_db = client['osu_mlpp_db1']
songs_path = "../../../data/2021_04_01_osu_files"

In [ ]:
beatmap_ids = osu_db.osu_beatmaps.distinct('_id')

In [ ]:
broken_paths = []
invalid_paths = []

for _id in (pbar := tqdm(beatmap_ids)):
    path = os.path.join(songs_path, f'{_id}.osu')
    query = {'_id': _id}
    
    update = {
        '$set': {
            'mlpp.hit_objects': None
        }
    }
    
    try:
        if os.path.exists(path):
            with open(path, encoding='latin-1') as file:
                update['$set']['mlpp.hit_objects'] = parse_hit_objects(file)
    except FileFormatError:
        invalid_paths.append(path)
    except Exception as e:
        broken_paths.append(path)
    
    osu_db.osu_beatmaps.update_one(query, update)
    
    pbar.set_description(f'(Errors: {len(broken_paths)} Invalid: {len(invalid_paths)}) ')

In [ ]:
N = osu_db.osu_beatmap_failtimes.count_documents({})
for doc in tqdm(osu_db.osu_beatmap_failtimes.find(), total=N):
    arr = [doc[f'p{i + 1}'] for i in range(100)]
    
    query = {
        '_id': doc['beatmap_id']
    }
    
    update = {
        '$set': {
            f'mlpp.{doc["type"]}': arr
        }
    }
    
    osu_db.osu_beatmaps.update_one(query, update)